# Home Credit Default Risk

Predicting how capable an applicant is of repaying a loan, using a dataset provided by Home Credit.

In [2]:
# Manipulating data
import numpy as np
import pandas as pd

# Managing files
import os

In [3]:
print(os.listdir("datasets/"))|

['bureau_balance.csv', 'credit_card_balance.csv', 'application_test.csv', 'POS_CASH_balance.csv', 'installments_payments.csv', 'bureau.csv', 'HomeCredit_columns_description.csv', 'previous_application.csv', 'application_train.csv', 'sample_submission.csv']


In [4]:
X_train = pd.read_csv('datasets/application_train.csv')
X_test = pd.read_csv('datasets/application_test.csv')